# HW- Visualizing features of a pretrained VGG
In this homework, we are going to try to visualize what neurons encode through optimization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

First, load the pretrained VGG16 model with torchvision, and print the architecture of the model. Describe the architecture and its composition.

In [ ]:
### YOUR CODE HERE ###

# Part A - Visualizing the convolution filters

First, plot all the filters for the red channel of the first convolutional layer (there should be 64 filters in total) using `model.features[i].weight` to access the weights of the i-th layer of the feature extraction part of the model. Can you find filters that seem to encode edges? Is this method useful for other layers?

In [ ]:
### YOUR CODE HERE ###

# Part B - Visualizing channel activations through image optimization
## B.1 - First implementation
Create a module `ChannelActivation(layer, channel)` that returns the average activation (i.e. output value) of channel `channel` of layer `layer` of the VGG features (both indexed starting from 0).

In [ ]:
### YOUR CODE HERE ###

Our objective is to find which patterns are recognized by a given channel. To do so, we will follow the approach of [this Distill article](https://distill.pub/2017/feature-visualization/) and find images that lead to the highest possible channel activation.

First, create a random (colored) image of size 128x128 (i.e. tensor of shape (3, 128, 128)), initialized with value at random between 0.4 and 0.6 (i.e. grey + small perturbation). Then, perform 200 steps of Adam (with lr=0.01) to maximize the activation of channel 4 of layer 1. Plot the image after 0, 10, 50, 100 and 200 iterations. You should see a pink or blue saturated image with several horizontal lines, indicating that the channel probably recognizes horizontal edges.

**NB1:** Careful, by default, optimizers minimize their objective, not maximize it!

**NB2:** Tensors are created, by default, with `requires_grad` set to False. To do optimize on such a tensor, first set this field to True.

**NB3:** The parameters given to an optimizer should be on the cpu. If you use a gpu, you thus need to keep two versions of the image: 1) a cpu version given to the optimizer, and 2) a gpu version, created at each iteration of the optimization with x.to(device), and used to compute the gradient.

In [ ]:
### YOUR CODE HERE ###

## B.2 - Improving stability with clipping and normalization
Compute the highest and lowest values of the image. What is the issue?

In [ ]:
### YOUR CODE HERE ###

To avoid (over) saturation, clip the image pixels to $[0.2,0.8]$ after each optimization step using `input_image.data = input_image.data.clip(0.2, 0.8)`. You should now see several clear horizontal lines in a blue background.

In [ ]:
### YOUR CODE HERE ###

One issue with our current code, is that VGG was trained on **normalized** images, and thus is not adapted to our input image. To normalize the image, we will use **transforms**.

Create a function `create_activation_image(layer, channel, transform=None, image_size=128, show_steps=False)` that maximizes the corresponding channel activation on an image of size `image_size`, and first applies `transform` to the image before computing the gradient of the activation. The function should return the final image after 200 steps, and plot intermediate images for the steps 0,10,50,100,200 if `show_steps=True`.

Then, test your function with `transform=transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])`. Is this better? You should now see a horizontal pattern with lines.

In [ ]:
### YOUR CODE HERE ###

Now test your function on channel 0 of layer 20. The pattern that appears should vagely resemble fish scales.

In [ ]:
### YOUR CODE HERE ###

## B.3 - Transformation robustness
Large neural network are prone to adversarial attacks, i.e. a small well-crafted additive noise can dramatically change the output of the model, and thus lead to incorrect classification. For our purpose, this is an issue, as the optimization algorithm may find such very specific noise instead of more valuable visual patterns.

To avoid this issue and further improve our images, we are thus going to apply small random perturbations to the image before computing the gradient. This will prevent the optimizer from optimizing the noise, and overall increase the stability of our process.

To do so, add a composition of several transforms (before the normalization):

1.   A small pixel noise with `transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1))`
2.   A random affine transform with `transforms.RandomAffine(degrees=5, translate=(0.1,0.1), scale=(0.9,1.1))`
3.   A random crop of size 96 (to reduce the size of the image)
4.   Random local fluctations with `transforms.ElasticTransform(alpha=50.)`

Compare the activation images with and without these random transformations. Is the pattern more visible?



In [ ]:
### YOUR CODE HERE ###

To see what the transformation is doing to an image, apply the random transformations (without normalization) to the following simple image, and show 5 randomly transformed images.

In [ ]:
sample_image = 0.3 * torch.ones(3, 256, 256)
sample_image[0,:,40:80] += 0.7
sample_image[1,10:20,:] += 0.5
sample_image[2,150:,:] += 0.5
plt.imshow(tensor_to_image(sample_image))

### YOUR CODE HERE ###

## B.4 - Final visualization
Finally, show the activation images for the first 5 channels of layers [1, 10, 20, 30]. You should be able to see a gradual complexification of the patterns.

**PS1:** Our method seems unable to find meaningful patterns for the last layer. One issue is probably that the random crop imposes that all regions on the image look similar (as they all should have a high channel activation), thus preventing larger and more complex patterns to emerge from the optimization.

**PS2:** You can also try other layers and channels to find interesting patterns!

In [ ]:
### YOUR CODE HERE ###